In [ ]:
#limpieza de entorno(eliminacion de archivos)
!rm -rf hadoop-2.7.7 pig-0.17.0 apache-hive-1.2.2-bin
!rm -rf metastore_db derby.log output_pig coca_pig_final.csv resumen_hive.txt
!rm -f *.tar.gz *.xlsx
!rm -f Clasificacion_Funcional_Gasto_mef.csv

In [1]:
print("--------1 DESCARGA DE LIBRERIAS--------")

#importamalos la librerias OS y PANDAS
import os
import pandas as pd

#LIBRERIA Q PERMITA LA DESCARGA Y DESCARGA DE ARCHIVOS
from google.colab import files

#DESCARGAMOS UNA LIBRERIA PARA DESCARGAR LOS ARCHIVOS EN .XML(EXCEL)
!pip install -q openpyxl xlsxwriter

print("--------2 COFIGURACION DE JAVA 8--------")

#COFIGURACION DE JAVA 8
!apt-get update -qq
!apt-get install -y openjdk-8-jdk-headless -qq

os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"]= os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]


print("--------4 DESCARGA E INSTALACIÓN DE HADOOP, PIG Y HIVE--------")
#6 DESCARGA E INSTALACIÓN DE HADOOP, PIG Y HIVE
!wget --progress=bar:force https://archive.apache.org/dist/hadoop/core/hadoop-2.7.7/hadoop-2.7.7.tar.gz
!wget --progress=bar:force https://archive.apache.org/dist/pig/pig-0.17.0/pig-0.17.0.tar.gz
!wget --progress=bar:force https://archive.apache.org/dist/hive/hive-1.2.2/apache-hive-1.2.2-bin.tar.gz

!tar -xzf hadoop-2.7.7.tar.gz
!tar -xzf pig-0.17.0.tar.gz
!tar -xzf apache-hive-1.2.2-bin.tar.gz

os.environ["HADOOP_HOME"]= "/content/hadoop-2.7.7"
os.environ["PIG_HOME"]= "/content/pig-0.17.0"
os.environ["HIVE_HOME"]= "/content/apache-hive-1.2.2-bin"

os.environ["PATH"] += (
      ":" + os.environ["HADOOP_HOME"] + "/bin"
      ":" + os.environ["PIG_HOME"] + "/bin"
      ":" + os.environ["HIVE_HOME"] + "/bin"
)



--------1 DESCARGA DE LIBRERIAS--------
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 2.5 MB/s eta 0:00:00
--------2 COFIGURACION DE JAVA 8--------
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 117540 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u482-ga~us1-0ubuntu1~22.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u482-ga~us1-0ubuntu1~22.04) ...
Selecting previously unselected package openjdk-8-jdk-headless:amd64.
Preparing to unpack .../openjdk-8-jdk-headless_8u482-ga~us1-0ubuntu1~22.04_amd64.deb ...
Unpacking openjdk-

In [6]:
!rm -f Clasificacion_Funcional_Gasto_mef.csv

In [7]:
import pandas as pd
import time
import warnings

# Ocultar los avisos de pandas (ParserWarning)
warnings.filterwarnings('ignore', category=pd.errors.ParserWarning)

print("-------- DESCARGA DE LOS DATOS DEL MINISTERIO DE ECONOMÍA Y FINANZAS --------")

# 1. Descarga
inicio = time.time()
print("--- Descargando datos del MEF ---")
!wget -q -O Clasificacion_Funcional_Gasto_mef.csv https://fs.datosabiertos.mef.gob.pe/datastorefiles/EP_Clasificacion_Funcional_Gasto.csv
fin = time.time()

# 2. Lectura y conteo
try:
    # Leemos el archivo silenciando los errores de filas mal formadas
    df = pd.read_csv('Clasificacion_Funcional_Gasto_mef.csv',
                     encoding='latin-1',
                     on_bad_lines='skip',
                     quoting=3)

    # 3. Impresión de resultados limpios
    print(f"✅ Descarga y procesamiento completado.")
    print(f"⏱️  Tiempo transcurrido: {fin - inicio:.2f} segundos")
    print(f"📊 Total de registros leídos: {len(df):,}")

except Exception as e:
    print(f"❌ Error al leer los datos: {e}")

-------- DESCARGA DE LOS DATOS DEL MINISTERIO DE ECONOMÍA Y FINANZAS --------
--- Descargando datos del MEF ---
✅ Descarga y procesamiento completado.
⏱️  Tiempo transcurrido: 22.98 segundos
📊 Total de registros leídos: 311,835


In [14]:
!rm -f etl_Gasto.pig
!rm -f output_full_data
!rm -f data_gato_maestra_final.csv

In [15]:
print("--------PROCESO Y LIMPIEZA ETL CON PIG--------")


with open("etl_Gasto.pig","w") as f:
  f.write("""
Clasificacion_Gasto_raw= LOAD 'Clasificacion_Funcional_Gasto_mef.csv' USING PigStorage(',') as (
  ano:chararray, mes:chararray, nivel_gob_nom:chararray, nivel:chararray, nivel_nom:chararray,
  pliego:chararray, pliego_nom:chararray, depto_cod:chararray, depto_nom:chararray, prov_cod:chararray,
  prov_nom:chararray, dis_cod:chararray, dis_nom:chararray, serv_cod:chararray, serv_nom:chararray,
  func_cod:chararray,func_nom:chararray,monto_pim:chararray,monto_ejec:chararray
);

--FILTRO-QUITAMOS CABECERA
Gasto_filtro = FILTER Clasificacion_Gasto_raw BY NOT(ano MATCHES '.*ANO.*');

--LIMPIEZA-QUITAMOS LAS COMILLAS A TODO Y PONEMOS LOS VALORES EN NUMEROS A LOS Q LE PERTENECE
Data_Gasto_final= FOREACH Gasto_filtro GENERATE
    REPLACE(ano, '"', '') AS ano,
    REPLACE(mes, '"', '') AS mes,
    REPLACE(nivel_gob_nom, '"', '') AS nivel_gobierno,
    REPLACE(pliego_nom, '"', '') AS pliego,
    REPLACE(depto_nom, '"', '') AS departamento,
    REPLACE(func_nom, '"', '') AS funcion_gasto,
    (double)REPLACE(REPLACE(monto_pim, '"', ''), ',', '') AS pim,
    (double)REPLACE(monto_ejec, '"', '') AS ejecucion;

-- Guardamos la data completa sin agrupar
  STORE Data_Gasto_final INTO 'output_full_data' USING PigStorage(',');

""")
# 2. Iniciar el cronómetro
inicio_etl = time.time()

print("--- Ejecutando proceso ETL con Apache Pig (esto puede tardar...) ---")
# Ejecutar Pig (usamos -brief para reducir un poco el log innecesario si está disponible)
!pig -x local etl_Gasto.pig

print("--- Unificando archivos de salida ---")
!cat output_full_data/part-* > data_gato_maestra_final.csv

# 3. Calcular tiempo final
fin_etl = time.time()
duracion_etl = fin_etl - inicio_etl

print(f"\n✅ Proceso ETL finalizado con éxito.")
print(f"⏱️  Tiempo total del proceso Pig: {duracion_etl:.2f} segundos")

--------PROCESO Y LIMPIEZA ETL CON PIG--------
--- Ejecutando proceso ETL con Apache Pig (esto puede tardar...) ---
26/02/16 17:58:47 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
26/02/16 17:58:47 INFO pig.ExecTypeProvider: Picked LOCAL as the ExecType
2026-02-16 17:58:47,709 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2026-02-16 17:58:47,709 [main] INFO  org.apache.pig.Main - Logging error messages to: /content/pig_1771264727707.log
2026-02-16 17:58:47,734 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - user.name is deprecated. Instead, use mapreduce.job.user.name
2026-02-16 17:58:48,149 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2026-02-16 17:58:48,319 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2026-02-16 17:58:48,321 [main] INFO  org.apache.pig.backend.hadoop

In [16]:
import pandas as pd

# 1. Definimos los nombres reales de las columnas
nombres_columnas = [
    'ano', 'mes', 'nivel_gobierno', 'pliego',
    'departamento', 'funcion_gasto', 'pim', 'ejecucion'
]

input_file = 'data_gato_maestra_final.csv'

try:
    # Leemos indicando que NO hay cabecera (header=None) y asignando los nombres manualmente
    df = pd.read_csv(input_file, header=None, names=nombres_columnas, low_memory=False)

    # Si por casualidad la primera fila fuera texto (cabecera accidental), la quitamos
    df = df[df['ano'].astype(str).str.contains(r'^\d+$')]

    # 2. Convertir montos a números (limpieza profunda)
    df['pim'] = pd.to_numeric(df['pim'], errors='coerce')
    df['ejecucion'] = pd.to_numeric(df['ejecucion'], errors='coerce')

    # 3. Eliminar filas basura (montos vacíos o años que no son de 4 dígitos)
    df = df.dropna(subset=['pim', 'ejecucion', 'ano'])
    df = df[df['ano'].astype(str).str.len() == 4]

    # 4. Limpiar textos para que Hive no se confunda con mayúsculas/minúsculas
    for col in ['departamento', 'funcion_gasto']:
        df[col] = df[col].astype(str).str.strip().str.upper()

    # 5. Guardar el archivo final PERFECTO y SIN CABECERA
    df.to_csv('gasto_limpio_final.csv', index=False, header=False)

    print("✅ ¡Éxito! Archivo 'gasto_limpio_final.csv' generado.")
    print(f"Filas procesadas: {len(df)}")
    print("\nVista previa de los datos limpios:")
    print(df.head(3))

except Exception as e:
    print(f"❌ Error crítico: {e}")

✅ ¡Éxito! Archivo 'gasto_limpio_final.csv' generado.
Filas procesadas: 282429

Vista previa de los datos limpios:
    ano  mes     nivel_gobierno                                 pliego  \
0  2018   13  GOBIERNOS LOCALES  MUNICIPALIDAD DISTRITAL DE CAPACMARCA   
1  2018   13  GOBIERNOS LOCALES  MUNICIPALIDAD DISTRITAL DE CAPACMARCA   
2  2018   13  GOBIERNOS LOCALES  MUNICIPALIDAD DISTRITAL DE CAPACMARCA   

  departamento funcion_gasto        pim  ejecucion  
0        CUSCO      AMBIENTE   433306.0  403935.00  
1        CUSCO   SANEAMIENTO  2665371.0  578282.61  
2        CUSCO         SALUD   509125.0   28315.50  


In [17]:
# 1. Creamos la cabecera en un archivo temporal
!echo "ano,mes,nivel_gobierno,pliego,departamento,funcion_gasto,pim,ejecucion" > temporal_header.csv

# 2. Unimos la cabecera con el archivo limpio que generó Pandas
# IMPORTANTE: Usamos '>' para crear un archivo NUEVO, no sobrescribimos el anterior
!cat temporal_header.csv gasto_limpio_final.csv > data_maestra_con_cabecera.csv

# 3. Limpiamos el temporal
!rm temporal_header.csv

print("✅ Archivos listos:")
print("- gasto_limpio_final.csv (Para Hive - Sin cabecera)")
print("- data_maestra_con_cabecera.csv (Para MapReduce - Con cabecera)")

✅ Archivos listos:
- gasto_limpio_final.csv (Para Hive - Sin cabecera)
- data_maestra_con_cabecera.csv (Para MapReduce - Con cabecera)


In [ ]:
#------------------------------------------------
#------------------------------------------------
#        LIMPIEZA
#------------------------------------------------
#------------------------------------------------
# Limpiar HDFS
!hdfs dfs -rm -r /output_presupuesto_dept 2>/dev/null


# Limpiar archivos locales
!rm -f mapper.py reducer.py resultado_dept.txt data_gato_maestra_final.csv
#------------------------------------------------
#------------------------------------------------

Deleted /output_presupuesto_dept


In [18]:
from google.colab import files
import pandas as pd

# -----------------------------------------------------
# 1. PREPARACIÓN: Subir el archivo limpio a HDFS
# -----------------------------------------------------
!hdfs dfs -rm -f /data_maestra_con_cabecera.csv
!hdfs dfs -put data_maestra_con_cabecera.csv /

# -----------------------------------------------------
# 2. CREAR MAPPER (Extrae Departamento, PIM y Ejecución)
# -----------------------------------------------------

mapper_code = """import sys
import csv

# Leemos desde la entrada estándar (Hadoop envía el archivo línea por línea)
reader = csv.reader(sys.stdin, delimiter=',')

for fields in reader:
    # Saltamos líneas vacías o la cabecera
    if not fields or len(fields) < 8 or fields[0].lower().startswith('ano'):
        continue

    try:
        # Según tu archivo maestro 'data_gato_maestra_final.csv':
        # col 4: departamento, col 6: pim, col 7: ejecucion
        departamento = fields[4].strip().upper()
        pim = fields[6].replace(',', '').strip()
        ejecucion = fields[7].replace(',', '').strip()

        if departamento and pim and ejecucion:
            # MAP: Emitimos llave (Dpto) y valores (PIM|Ejec)
            print(f"{departamento}\\t{pim}|{ejecucion}")
    except (IndexError, ValueError):
        continue
"""
with open("mapper.py", "w") as f: f.write(mapper_code)

# -----------------------------------------------------
# 3. CREAR REDUCER (Cálculo de Totales y Brecha)
# -----------------------------------------------------
reducer_code = """import sys

current_dept = None
current_pim = 0.0
current_ejec = 0.0

for line in sys.stdin:
    line = line.strip()
    if not line: continue

    try:
        dept, values = line.split("\\t")
        pim, ejec = values.split("|")
        pim = float(pim)
        ejec = float(ejec)

        # REDUCE: Agregación por llave (Departamento)
        if current_dept == dept:
            current_pim += pim
            current_ejec += ejec
        else:
            if current_dept:
                # Emitimos: Depto, PIM total, Ejecución Total, Saldo (Brecha)
                brecha = current_pim - current_ejec
                print(f"{current_dept}\\t{current_pim:.2f}\\t{current_ejec:.2f}\\t{brecha:.2f}")
            current_dept = dept
            current_pim = pim
            current_ejec = ejec

    except ValueError:
        continue

# No olvidar procesar el último grupo
if current_dept:
    brecha = current_pim - current_ejec
    print(f"{current_dept}\\t{current_pim:.2f}\\t{current_ejec:.2f}\\t{brecha:.2f}")
"""
with open("reducer.py", "w") as f: f.write(reducer_code)

# -----------------------------------------------------
# 4. EJECUTAR EN HADOOP (Streaming)
# -----------------------------------------------------

!hdfs dfs -rm -r /output_presupuesto 2>/dev/null

!hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
-files mapper.py,reducer.py \
-mapper "python3 mapper.py" \
-reducer "python3 reducer.py" \
-input /data_maestra_con_cabecera.csv \
-output /output_presupuesto

# -----------------------------------------------------
# 5. MOSTRAR RESULTADOS
# -----------------------------------------------------
!hdfs dfs -cat /output_presupuesto/part-* > resultado_ejec.txt

fd = pd.read_csv("resultado_ejec.txt", sep="\\t", header=None,
                 names=["DEPARTAMENTO", "PIM_TOTAL", "EJECUCION_TOTAL", "SALDO_BRECHA"])

# Cálculo de % de Avance
fd['%_AVANCE'] = (fd['EJECUCION_TOTAL'] / fd['PIM_TOTAL']) * 100
fd = fd.sort_values("SALDO_BRECHA", ascending=False).head(10) # Top 10 brechas más grandes

display(fd)

26/02/16 17:59:56 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
26/02/16 17:59:56 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
26/02/16 17:59:56 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
26/02/16 17:59:56 INFO mapred.FileInputFormat: Total input paths to process : 1
26/02/16 17:59:56 INFO mapreduce.JobSubmitter: number of splits:1
26/02/16 17:59:56 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1741949530_0001
26/02/16 17:59:57 INFO mapred.LocalDistributedCacheManager: Localized file:/content/mapper.py as file:/tmp/hadoop-root/mapred/local/1771264796953/mapper.py
26/02/16 17:59:57 INFO mapred.LocalDistributedCacheManager: Localized file:/content/reducer.py as file:/tmp/hadoop-root/mapred/local/1771264796954/reducer.py
26/02/16 17:59:57 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
26/02/1

,DEPARTAMENTO,PIM_TOTAL,EJECUCION_TOTAL,SALDO_BRECHA,%_AVANCE
13,LIMA,2.002571e+12,1.865091e+12,1.374798e+11,93.134837
1,ANCASH,5.990226e+10,4.156445e+10,1.833781e+10,69.387119
3,AREQUIPA,8.007164e+10,6.375586e+10,1.631578e+10,79.623519
6,CUSCO,9.090678e+10,7.489996e+10,1.600682e+10,82.392048
11,LA LIBERTAD,7.708157e+10,6.289068e+10,1.419089e+10,81.589775
5,CAJAMARCA,5.887935e+10,4.590852e+10,1.297082e+10,77.970503
18,PIURA,8.869011e+10,7.852047e+10,1.016964e+10,88.533511
10,JUNIN,6.357412e+10,5.521257e+10,8.361557e+09,86.847547
20,PUNO,5.132650e+10,4.300038e+10,8.326112e+09,83.778140
4,AYACUCHO,3.893030e+10,3.099130e+10,7.938999e+09,79.607145


In [ ]:
# -----------------------------------------------------
# 6. LIMPIEZA DE ARCHIVOS GENERADOS (MapReduce)
# -----------------------------------------------------
print("--- Iniciando limpieza de archivos temporales ---")

# 1. Eliminar archivos de script de Python locales
!rm -f mapper.py reducer.py

# 2. Eliminar archivos de resultados temporales locales
!rm -f  resultado_ejec.txt

# 3. Eliminar archivos y directorios en el sistema de archivos de Hadoop (HDFS)
!hdfs dfs -rm -r /output_presupuesto 2>/dev/null
!hdfs dfs -rm /data_maestra_hadoop.csv 2>/dev/null

print("--- Limpieza completada con éxito ---")

--- Iniciando limpieza de archivos temporales ---
Deleted /output_presupuesto
--- Limpieza completada con éxito ---


In [19]:
from google.colab import files
import pandas as pd
# -----------------------------------------------------
# 1. PREPARACIÓN: Subir el archivo limpio a HDFS
# -----------------------------------------------------
!hdfs dfs -rm -f /data_maestra_con_cabecera.csv
!hdfs dfs -put data_maestra_con_cabecera.csv /

# -----------------------------------------------------
# 2. CREAR MAPPER (Extrae Departamento, PIM y Ejecución)
# -----------------------------------------------------
mapper_code = """import sys
import csv

# Leemos desde la entrada estándar
reader = csv.reader(sys.stdin, delimiter=',')

for fields in reader:
    if not fields or fields[0].lower().startswith('ano'):
        continue

    try:
        # Según tu archivo maestro:
        # col 4: departamento, col 6: pim, col 7: ejecucion
        departamento = fields[4].strip().upper()
        pim = fields[6].strip()
        ejecucion = fields[7].strip()

        if departamento:
            # MAP: Emitimos llave (Dpto) y valores (PIM|Ejec)
            print(f"{departamento}\\t{pim}|{ejecucion}")
    except (IndexError, ValueError):
        continue
"""
with open("mapper.py", "w") as f: f.write(mapper_code)

# -----------------------------------------------------
# 3. CREAR REDUCER (Suma los totales por departamento)
# -----------------------------------------------------
reducer_code = """import sys

current_dept = None
current_pim = 0.0
current_ejec = 0.0

for line in sys.stdin:
    line = line.strip()
    if not line: continue

    try:
        dept, values = line.split("\\t")
        pim, ejec = values.split("|")
        pim = float(pim)
        ejec = float(ejec)

        # REDUCE: Agregación por llave
        if current_dept == dept:
            current_pim += pim
            current_ejec += ejec
        else:
            if current_dept:
                print(f"{current_dept}\\t{current_pim:.2f}\\t{current_ejec:.2f}")
            current_dept = dept
            current_pim = pim
            current_ejec = ejec

    except ValueError:
        continue

# No olvidar el último departamento
if current_dept:
    print(f"{current_dept}\\t{current_pim:.2f}\\t{current_ejec:.2f}")
"""
with open("reducer.py", "w") as f: f.write(reducer_code)

# -----------------------------------------------------
# 4. EJECUTAR EN HADOOP
# -----------------------------------------------------
!hdfs dfs -rm -r /output_presupuesto_dept 2>/dev/null

!hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
-files mapper.py,reducer.py \
-mapper "python3 mapper.py" \
-reducer "python3 reducer.py" \
-input /data_maestra_con_cabecera.csv \
-output /output_presupuesto_dept

# -----------------------------------------------------
# 5. MOSTRAR RESULTADOS
# -----------------------------------------------------
!hdfs dfs -cat /output_presupuesto_dept/part-* > resultado_dept.txt

import pandas as pd
df = pd.read_csv("resultado_dept.txt", sep="\\t", header=None,
                 names=["DEPARTAMENTO", "PIM_TOTAL", "EJECUCION_TOTAL"])
df['%_AVANCE'] = (df['EJECUCION_TOTAL'] / df['PIM_TOTAL']) * 100
df = df.sort_values("EJECUCION_TOTAL", ascending=False)

display(df)

26/02/16 18:00:24 INFO Configuration.deprecation: io.bytes.per.checksum is deprecated. Instead, use dfs.bytes-per-checksum
26/02/16 18:00:24 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /data_maestra_con_cabecera.csv
26/02/16 18:00:29 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
26/02/16 18:00:29 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
26/02/16 18:00:29 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
26/02/16 18:00:29 INFO mapred.FileInputFormat: Total input paths to process : 1
26/02/16 18:00:29 INFO mapreduce.JobSubmitter: number of splits:1
26/02/16 18:00:30 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1720675980_0001
26/02/16 18:00:30 INFO mapred.LocalDistributedCacheManager: Localized file:/content/mapper.py as file:/tmp/ha

,DEPARTAMENTO,PIM_TOTAL,EJECUCION_TOTAL,%_AVANCE
13,LIMA,2.002571e+12,1.865091e+12,93.134837
18,PIURA,8.869011e+10,7.852047e+10,88.533511
6,CUSCO,9.090678e+10,7.489996e+10,82.392048
3,AREQUIPA,8.007164e+10,6.375586e+10,79.623519
11,LA LIBERTAD,7.708157e+10,6.289068e+10,81.589775
10,JUNIN,6.357412e+10,5.521257e+10,86.847547
5,CAJAMARCA,5.887935e+10,4.590852e+10,77.970503
20,PUNO,5.132650e+10,4.300038e+10,83.778140
14,LORETO,4.731616e+10,4.261952e+10,90.073911
1,ANCASH,5.990226e+10,4.156445e+10,69.387119


In [20]:
import pandas as pd

mapper_code = """import sys
import csv

reader = csv.reader(sys.stdin, delimiter=',')

for fields in reader:
    if not fields:
      continue

    # Ignorar encabezado
    if fields[0].lower().startswith('ano_eje'):
        continue

    try:
        ano = fields[0].strip()       # Columna ANO_EJE
        pim = fields[6].strip()      # Columna AFIN_AP3
        ejec = fields[7].strip()     # Columna EJEC_AP3

        if ano:
            # Emitimos llave: Año, valor: PIM|Ejecucion
            print(f"{ano}\\t{pim}|{ejec}")
    except (IndexError, ValueError):
        continue
"""
open("mapper.py", "w").write(mapper_code)

reducer_code = """import sys
from collections import defaultdict

pim_por_ano = defaultdict(float)
ejec_por_ano = defaultdict(float)

for line in sys.stdin:
    try:
        line = line.strip()
        if not line: continue

        ano, valores = line.split("\\t")
        pim, ejec = valores.split("|")

        pim_por_ano[ano] += float(pim)
        ejec_por_ano[ano] += float(ejec)
    except ValueError:
        continue

# El framework de Hadoop ya entrega las llaves ordenadas al reducer,
# pero imprimimos el resultado acumulado:
for ano in sorted(pim_por_ano.keys()):
    print(f"{ano}\\t{pim_por_ano[ano]:.2f}\\t{ejec_por_ano[ano]:.2f}")
"""
open("reducer.py", "w").write(reducer_code)

# -----------------------------------------------------
# 4. EJECUTAR EN HADOOP
# -----------------------------------------------------
!hdfs dfs -rm -r /output_pim_ejec_ano 2>/dev/null

!hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
-files mapper.py,reducer.py \
-mapper "python3 mapper.py" \
-reducer "python3 reducer.py" \
-input /data_maestra_con_cabecera.csv \
-output /output_pim_ejec_ano

# -----------------------------------------------------
# 5. MOSTRAR RESULTADOS
# -----------------------------------------------------
!hdfs dfs -cat /output_pim_ejec_ano/part-* > resultado_año.txt
# -----------------------------------------------------
# 6. MOSTRAR RESULTADO COMO TABLA
# -----------------------------------------------------
dt = pd.read_csv(
    "resultado_año.txt",
    sep="\\t",
    header=None,
    names=["ano", "pim_total", "total_ejecucion"],
    engine="python"
)

# Ordenar por año (aunque el reducer ya lo hace, aseguramos en el DF)
dt = dt.sort_values("ano")

# Opcional: ver porcentaje de ejecución anual
dt['%_ejecucion'] = (dt['total_ejecucion'] / dt['pim_total']) * 100

display(dt)

26/02/16 18:00:54 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
26/02/16 18:00:54 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
26/02/16 18:00:54 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
26/02/16 18:00:54 INFO mapred.FileInputFormat: Total input paths to process : 1
26/02/16 18:00:54 INFO mapreduce.JobSubmitter: number of splits:1
26/02/16 18:00:54 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local2594513_0001
26/02/16 18:00:55 INFO mapred.LocalDistributedCacheManager: Localized file:/content/mapper.py as file:/tmp/hadoop-root/mapred/local/1771264855026/mapper.py
26/02/16 18:00:55 INFO mapred.LocalDistributedCacheManager: Localized file:/content/reducer.py as file:/tmp/hadoop-root/mapred/local/1771264855027/reducer.py
26/02/16 18:00:55 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
26/02/16 1

,ano,pim_total,total_ejecucion,%_ejecucion
0,2011,9.939638e+10,8.256807e+10,83.069497
1,2012,1.064622e+11,9.049559e+10,85.002558
2,2013,1.157108e+11,1.010933e+11,87.367234
3,2014,1.260561e+11,1.134309e+11,89.984465
4,2015,1.849326e+11,1.711514e+11,92.547997
5,2016,1.901792e+11,1.707436e+11,89.780366
6,2017,2.162428e+11,1.935981e+11,89.528125
7,2018,2.336738e+11,2.118673e+11,90.668001
8,2019,2.485034e+11,2.154075e+11,86.681912
9,2020,2.709945e+11,2.274441e+11,83.929439


In [21]:

# ===============================================================
# PROCESAMIENTO CON HIVE Y GENERACIÓN DE RESULTADOS
# ===============================================================
import os
import pandas as pd
from google.colab import files
!pip install -q openpyxl xlsxwriter
import pytz
from datetime import datetime

hive_sql= """
CREATE TABLE IF NOT EXISTS clasificacion_gasto(
  ano STRING,
  mes STRING,
  nivel_gobierno STRING,
  pliego STRING,
  departamento STRING,
  funcion_gasto STRING,
  pim DOUBLE,
  ejecucion DOUBLE
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
"""

# Reiniciar esquema y crear tabla
!hive -e "DROP TABLE IF EXISTS clasificacion_gasto"
!hive -e "{hive_sql}"

#Cargar el archivo SIN CABECERA
!hive -e "LOAD DATA LOCAL INPATH 'gasto_limpio_final.csv' OVERWRITE INTO TABLE clasificacion_gasto"

print("\n---DEPARTAMENTOS POR PIM Y EJECUCIÓN (AÑOS) ---")
hive_dep_año = !hive -S -e """SELECT departamento,ano, SUM(pim), SUM(ejecucion) AS total_ejecucion,(SUM(ejecucion) / SUM(pim)) * 100 AS porcentaje_ejecucion FROM clasificacion_gasto GROUP BY departamento,ano ORDER BY ano,departamento"""
for row in hive_dep_año: print(row)



Logging initialized using configuration in jar:file:/content/apache-hive-1.2.2-bin/lib/hive-common-1.2.2.jar!/hive-log4j.properties
OK
Time taken: 1.017 seconds

Logging initialized using configuration in jar:file:/content/apache-hive-1.2.2-bin/lib/hive-common-1.2.2.jar!/hive-log4j.properties
OK
Time taken: 1.804 seconds

Logging initialized using configuration in jar:file:/content/apache-hive-1.2.2-bin/lib/hive-common-1.2.2.jar!/hive-log4j.properties
Loading data to table default.clasificacion_gasto
Table default.clasificacion_gasto stats: [numFiles=1, numRows=0, totalSize=29332148, rawDataSize=0]
OK
Time taken: 2.913 seconds

---DEPARTAMENTOS POR PIM Y EJECUCIÓN (AÑOS) ---
AMAZONAS	2011	7.10621702E8	5.5433987216E8	78.00773190571655
ANCASH	2011	3.27235254E9	2.0697537606099985E9	63.24971821679086
APURIMAC	2011	9.28989679E8	6.538543669300011E8	70.38338333681328
AREQUIPA	2011	2.294367814E9	1.6840089222299993E9	73.39751333479957
AYACUCHO	2011	1.356738035E9	9.932602117199998E8	73.20943218

In [23]:
print("\n---NIVEL DE GOBIERNO POR PIM Y EJECUCIÓN ---")
hive_nivel_gob = !hive -S -e "SELECT ano,nivel_gobierno,SUM(pim) AS total_pim, SUM(ejecucion) AS total_ejecucion FROM clasificacion_gasto GROUP BY ano,nivel_gobierno ORDER BY ano ASC,nivel_gobierno ASC "
for row in hive_nivel_gob: print(row)

print("\n---PLIEGO POR PIM Y EJECUCIÓN (AÑO) ---")
hive_pliego_gob = !hive -S -e "SELECT ano,pliego,SUM(pim) AS total_pim, SUM(ejecucion) AS total_ejecucion FROM clasificacion_gasto GROUP BY ano,pliego ORDER BY ano ASC,pliego ASC"
for row in hive_pliego_gob: print(row)

print("\n---FUNCION POR PIM Y EJECUCIÓN (AÑO) ---")
hive_funcion = !hive -S -e "SELECT ano,funcion_gasto,SUM(pim) AS total_pim, SUM(ejecucion) AS total_ejecucion,(SUM(ejecucion) / SUM(pim)) * 100 AS porcentaje_ejecucion FROM clasificacion_gasto GROUP BY ano,funcion_gasto ORDER BY ano ASC,funcion_gasto ASC"
for row in hive_funcion: print(row)



Se han truncado las últimas 5000 líneas del flujo de salida.
2022	MUNICIPALIDAD PROVINCIAL DE PURUS	3272264.0	2038981.5
2022	MUNICIPALIDAD PROVINCIAL DE PUTUMAYO	1038058.0	994548.57
2022	MUNICIPALIDAD PROVINCIAL DE QUISPICANCHIS - URCOS	2.9885689E7	2.2578267840000004E7
2022	MUNICIPALIDAD PROVINCIAL DE RECUAY	2.16681E7	1.457993696E7
2022	MUNICIPALIDAD PROVINCIAL DE REQUENA	2.3636016E7	1.6564760119999997E7
2022	MUNICIPALIDAD PROVINCIAL DE RIOJA	2.6656721E7	2.000530136E7
2022	MUNICIPALIDAD PROVINCIAL DE RODRIGUEZ DE MENDOZA - SAN NICOLAS	4511292.0	3388961.51
2022	MUNICIPALIDAD PROVINCIAL DE SAN ANTONIO DE PUTINA	3.0514998E7	2.0716009439999998E7
2022	MUNICIPALIDAD PROVINCIAL DE SAN IGNACIO	1.12012495E8	8.432189271E7
2022	MUNICIPALIDAD PROVINCIAL DE SAN MARCOS - PEDRO GALVEZ	1.8242796E7	1.4540699330000002E7
2022	MUNICIPALIDAD PROVINCIAL DE SAN MARTIN - TARAPOTO	4.3309792E7	2.983927717E7
2022	MUNICIPALIDAD PROVINCIAL DE SAN MIGUEL	3.9460463E7	3.503726932E7
2022	MUNICIPALIDAD PROVINCIAL DE SA

In [24]:
import pandas as pd
import pytz
from datetime import datetime
!pip install -q openpyxl xlsxwriter

# ===============================================================
# EXPORTACIÓN A EXCEL CON METADATOS Y RESULTADOS
# ===============================================================
# 1. Preparar metadatos (Hora Perú)
peru_tz = pytz.timezone("America/Lima")
fecha_peru = datetime.now(peru_tz).strftime("%Y-%m-%d %H:%M:%S")

info = {
    "Concepto": ["Fecha y Hora Generación (Perú)",
                 "Herramientas Usadas"
    ],
    "Valor": [fecha_peru,
              "HIVE"
    ]
}
df_info = pd.DataFrame(info)

# 2. Procesar los resultados de Hive para el Excel
# Convertimos las listas de Hive a DataFrames
df_hive_dep_año = pd.DataFrame([row.split('\t') for row in hive_dep_año], columns=['DEPARTAMENTO', 'AÑO','TOTAL_PIM','TOTAL_EJECUCION','%EJECUCION'])
df_hive_nivel_gob = pd.DataFrame([row.split('\t') for row in hive_nivel_gob], columns=['AÑO', 'NIVEL_GOB','TOTAL_PIM','TOTAL_EJECUCION'])
df_hive_pliego_gob = pd.DataFrame([row.split('\t') for row in hive_pliego_gob], columns=['AÑO', 'PLIEGO','TOTAL_PIM','TOTAL_EJECUCION'])
df_hive_funcion = pd.DataFrame([row.split('\t') for row in hive_funcion], columns=['AÑO', 'FUNCION','TOTAL_PIM','TOTAL_EJECUCION','%EJECUCION'])

# 3. Crear el archivo Excel
nombre_archivo = 'Reporte_Gasto_HIVE.xlsx'

with pd.ExcelWriter(nombre_archivo, engine='xlsxwriter') as writer:
   # Hoja 1: Info del Proceso
    df_info.to_excel(writer, sheet_name='RESUMEN_EJECUTIVO', index=False)

    df_hive_dep_año.to_excel(writer, sheet_name='HIVE_DEP_AÑOS', index=False)
    df_hive_nivel_gob.to_excel(writer, sheet_name='HIVE_NIVEL_GOB', index=False)
    df_hive_pliego_gob.to_excel(writer, sheet_name='HIVE_PLIEGO', index=False)
    df_hive_funcion.to_excel(writer, sheet_name='HIVE_FUNCION', index=False)

  # --- Estética del Excel ---
    workbook = writer.book
    formato_cabecera = workbook.add_format({'bold': True, 'bg_color': '#4F81BD', 'font_color': 'white', 'border': 1})

    for sheet in writer.sheets.values():
        sheet.set_column('A:H', 25) # Ajustar ancho de columnas

print(f"✅ ¡Excelente! El reporte '{nombre_archivo}' ha sido generado.")

# Descargar automáticamente
files.download(nombre_archivo)

✅ ¡Excelente! El reporte 'Reporte_Gasto_HIVE.xlsx' ha sido generado.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
import pandas as pd
import pytz
from datetime import datetime
!pip install -q openpyxl xlsxwriter

# 1. Preparar metadatos (Hora Perú)
peru_tz = pytz.timezone("America/Lima")
fecha_peru = datetime.now(peru_tz).strftime("%Y-%m-%d %H:%M:%S")

info = {
    "Concepto": ["Fecha y Hora Generación (Perú)",
                 "Herramientas Usadas"
    ],
    "Valor": [fecha_peru,
              "MapReduce"
    ]
}
df_info = pd.DataFrame(info)

# 3. Crear el archivo Excel
nombre_archivo = 'Reporte_Gasto_MapReduce.xlsx'

with pd.ExcelWriter(nombre_archivo, engine='xlsxwriter') as writer:

    df_info.to_excel(writer, sheet_name='RESUMEN_EJECUTIVO', index=False)

    # Hoja 1: Resultados MapReduce
    df.to_excel(writer, sheet_name='MAPREDUCE_DEPTOS', index=False)

    #Hoja 2: Resultados MapReduce
    fd.to_excel(writer, sheet_name='MAPREDUCE_DEPTOS2', index=False)

    # Hoja 1: Resultados MapReduce
    dt.to_excel(writer, sheet_name='MAPREDUCE_DEPTOS3', index=False)

  # --- Estética del Excel ---
    workbook = writer.book
    formato_cabecera = workbook.add_format({'bold': True, 'bg_color': '#4F81BD', 'font_color': 'white', 'border': 1})

    for sheet in writer.sheets.values():
        sheet.set_column('A:H', 25) # Ajustar ancho de columnas

print(f"✅ ¡Excelente! El reporte '{nombre_archivo}' ha sido generado.")

# Descargar automáticamente
files.download(nombre_archivo)

✅ ¡Excelente! El reporte 'Reporte_Gasto_MapReduce2.xlsx' ha sido generado.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
!pip install -q openpyxl xlsxwriter
import pandas as pd
import math
from datetime import datetime
from google.colab import files



# 1. Cargar el archivo procesado
df = pd.read_csv('data_maestra_con_cabecera.csv')

#Preparar metadatos (Hora Perú)
peru_tz = pytz.timezone("America/Lima")
fecha_peru = datetime.now(peru_tz).strftime("%Y-%m-%d %H:%M:%S")

info = {
    "Concepto": ["Fecha y Hora Generación (Perú)",
                 "DOCUMENTO"
    ],
    "Valor": [fecha_peru,
              "DATA MAESTRA"
    ]
}
df_info = pd.DataFrame(info)

# 2. Dividir el DataFrame en dos partes
mitad = math.ceil(len(df) / 2)
parte1 = df.iloc[:mitad]
parte2 = df.iloc[mitad:]

# 3. Exportar a un solo archivo Excel con dos hojas
nombre_archivo = 'Data_Gasto_MEF_Dividido.xlsx'

with pd.ExcelWriter(nombre_archivo) as writer:
    df_info.to_excel(writer, sheet_name='DATA MAESTRA EN PARTES', index=False)
    parte1.to_excel(writer, sheet_name='Parte 1', index=False)
    parte2.to_excel(writer, sheet_name='Parte 2', index=False)

    # --- Estética del Excel ---
    workbook = writer.book
    formato_cabecera = workbook.add_format({'bold': True, 'bg_color': '#4F81BD', 'font_color': 'white', 'border': 1})

    for sheet in writer.sheets.values():
        sheet.set_column('A:H', 25) # Ajustar ancho de columnas


# 4. Descargar el archivo
files.download(nombre_archivo)

print(f"Proceso finalizado. Se ha creado '{nombre_archivo}' con dos hojas.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Proceso finalizado. Se ha creado 'Data_Gasto_MEF_Dividido.xlsx' con dos hojas.
